### diabates 당뇨병 분류를 CNN으로 구현

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
url = "C:/Users/dnwjd/OneDrive/Desktop/CSE_6/딥러닝/AI-class-main/AI-class-main/diabetes.csv"
data = pd.read_csv(url)

In [3]:
X = data.drop('Outcome', axis=1)
y = data['Outcome']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# CNN 모델을 위한 데이터 형태 변환 (2차원)
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [7]:
# CNN 모델 생성
model = Sequential([
    Conv1D(32, 2, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # 이진 분류
])

C:\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# 모델 컴파일
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
# 모델 학습
model.fit(X_train_cnn, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.5467 - loss: 0.6779 - val_accuracy: 0.7073 - val_loss: 0.6348
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7311 - loss: 0.5878 - val_accuracy: 0.7073 - val_loss: 0.5804
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7549 - loss: 0.5445 - val_accuracy: 0.7236 - val_loss: 0.5527
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7407 - loss: 0.5188 - val_accuracy: 0.7317 - val_loss: 0.5328
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7617 - loss: 0.5105 - val_accuracy: 0.6992 - val_loss: 0.5219
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7688 - loss: 0.4832 - val_accuracy: 0.7317 - val_loss: 0.5071
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7549 - loss: 0.5104 - val_accuracy: 0.7480 - val_loss: 0.4928
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7770 - loss: 0.4654 - val_accuracy: 0.7480 - val_los

In [10]:
# 모델 평가
loss, accuracy = model.evaluate(X_test_cnn, y_test)
print(f"\nTest Accuracy: {accuracy:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7567 - loss: 0.5000 

Test Accuracy: 0.7727


In [11]:
# 모델 저장
model.save('diabetes_cnn_model.h5')
print("\nModel saved as 'diabetes_cnn_model.h5'.")


Model saved as 'diabetes_cnn_model.h5'.


### 저장한 모델을 사전학습 모델로 불러와 층 추가 후 Fine-tunning

In [12]:
# 사전 학습된 모델 불러오기
loaded_model = load_model('diabetes_cnn_model.h5')
print("\nPre-trained model loaded.")


Pre-trained model loaded.


In [17]:
# 기존 모델의 레이어를 고정하여 새로운 층이 추가될 때 기존 가중치가 변경되지 않도록 설정
for layer in loaded_model.layers:
    layer.trainable = False

In [18]:
# 새로운 모델에 사전 학습된 모델의 모든 레이어 추가
fine_tuned_model = Sequential()
for layer in loaded_model.layers:
    fine_tuned_model.add(layer)

In [19]:
# 추가적인 층 및 출력층 추가
fine_tuned_model.add(Dense(32, activation='relu'))  # 새로운 층 추가
fine_tuned_model.add(Dropout(0.5))  # 드롭아웃 추가로 과적합 방지
fine_tuned_model.add(Dense(1, activation='sigmoid'))  # 이진 분류용 출력층

In [20]:
# 모델 컴파일 (학습률을 낮게 설정하여 미세 조정)
fine_tuned_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
# 추가 학습 (Fine-Tuning)
fine_tuned_model.fit(X_train_cnn, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.4264 - loss: 0.6963 - val_accuracy: 0.4146 - val_loss: 0.6886
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5241 - loss: 0.6852 - val_accuracy: 0.5203 - val_loss: 0.6878
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5492 - loss: 0.6909 - val_accuracy: 0.5772 - val_loss: 0.6870
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5602 - loss: 0.6893 - val_accuracy: 0.6098 - val_loss: 0.6863
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5443 - loss: 0.6924 - val_accuracy: 0.6829 - val_loss: 0.6856
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5865 - loss: 0.6931 - val_accuracy: 0.7398 - val_loss: 0.6850
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6186 - loss: 0.6884 - val_accuracy: 0.7561 - val_loss: 0.6843
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6836 - loss: 0.6808 - val_accuracy: 0.7724 - val_los

In [22]:
# 미세 조정 후 모델 평가
loss, accuracy = fine_tuned_model.evaluate(X_test_cnn, y_test)
print(f"\nFine-Tuned Model Test Accuracy: {accuracy:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6938 - loss: 0.6842 

Fine-Tuned Model Test Accuracy: 0.7338
